# Text Classification

## Imports

In [43]:
!pip install spektral > /dev/null

import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

from spektral.data.loaders import SingleLoader
from spektral.datasets.citation import Citation
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

from spektral.layers import GCNConv
from spektral.models.gcn import GCN
from spektral.transforms import LayerPreprocess


## Model

### GCN Model

In [44]:
def mask_to_weights(mask):
    return mask.astype(np.float32) / np.count_nonzero(mask)


def create_GCN_text_classification(dataset, learning_rate):
  weights_tr, weights_va, weights_te = (
      mask_to_weights(mask)
      for mask in (dataset.mask_tr, dataset.mask_va, dataset.mask_te)
  )

  model = GCN(n_labels=dataset.n_labels)
  model.compile(
      optimizer=Adam(learning_rate),
      loss=CategoricalCrossentropy(reduction="sum"),
      weighted_metrics=["acc"],
  )
  return model, weights_tr, weights_va, weights_te

### Simplify GCN Model

In [45]:
class SGCN:
    def __init__(self, K):
        self.K = K

    def __call__(self, graph):
        out = graph.a
        for _ in range(self.K - 1):
            out = out.dot(out)
        out.sort_indices()
        graph.a = out
        return graph

In [46]:
def create_SGCN_text_classification(database, learning_rate):
  l2_reg = 5e-6
  a_dtype = dataset[0].a.dtype
  N = dataset.n_nodes  # Number of nodes in the graph
  F = dataset.n_node_features  # Original size of node features
  n_out = dataset.n_labels  # Number of classes

  # Model definition
  x_in = Input(shape=(F,))
  a_in = Input((N,), sparse=True, dtype=a_dtype)

  output = GCNConv(
      n_out, activation="softmax", kernel_regularizer=l2(l2_reg), use_bias=False
  )([x_in, a_in])

  # Build model
  model = Model(inputs=[x_in, a_in], outputs=output)
  optimizer = Adam(learning_rate=learning_rate)
  model.compile(
      optimizer=optimizer, loss="categorical_crossentropy", weighted_metrics=["acc"]
  )
  model.summary()
  return model

## Cora

In [47]:
learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "cora"
tf.random.set_seed(seed=seed)

#### GCN Model

In [48]:
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

Pre-processing node features


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


##### Train Model

In [49]:
model, weights_tr, weights_va, weights_te = create_GCN_text_classification(dataset, learning_rate)
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.9543 - acc: 0.1214 - val_loss: 1.9510 - val_acc: 0.2440
Epoch 2/200
1/1 [==============================] - 0s 145ms/step - loss: 1.9485 - acc: 0.2786 - val_loss: 1.9477 - val_acc: 0.4140
Epoch 3/200
1/1 [==============================] - 0s 152ms/step - loss: 1.9424 - acc: 0.5643 - val_loss: 1.9443 - val_acc: 0.4880
Epoch 4/200
1/1 [==============================] - 0s 108ms/step - loss: 1.9368 - acc: 0.5929 - val_loss: 1.9407 - val_acc: 0.5080
Epoch 5/200
1/1 [==============================] - 0s 80ms/step - loss: 1.9276 - acc: 0.6643 - val_loss: 1.9373 - val_acc: 0.4680
Epoch 6/200
1/1 [==============================] - 0s 84ms/step - loss: 1.9222 - acc: 0.5786 - val_loss: 1.9340 - val_acc: 0.4600
Epoch 7/200
1/1 [==============================] - 0s 78ms/step - loss: 1.9150 - acc: 0.6000 - val_loss: 1.9309 - val_acc: 0.4440
Epoch 8/200
1/1 [==============================] - 0s 85ms/step - loss: 1.9078 - acc: 0.5

##### Evaluate

In [50]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 21ms/step - loss: 1.0495 - acc: 0.8060
Done.
Test loss: 1.04948890209198
Test accuracy: 0.8059999346733093


#### SimpleGCN Model

In [51]:
K = 2  # Propagation steps for SGCN
dataset = Citation("cora", transforms=[LayerPreprocess(GCNConv), SGCN(K)])
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


##### Train model

In [52]:
model = create_SGCN_text_classification(dataset, learning_rate)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 1433)]               0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 2708)]               0         []                            
                                                                                                  
 gcn_conv_14 (GCNConv)       (None, 7)                    10031     ['input_5[0][0]',             
                                                                     'input_6[0][0]']             
                                                                                                  
Total params: 10031 (39.18 KB)
Trainable params: 10031 (39.18 KB)
Non-trainable params: 0 (0

In [53]:
loader_tr = SingleLoader(dataset, sample_weights=mask_tr)
loader_va = SingleLoader(dataset, sample_weights=mask_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 1s 837ms/step - loss: 0.1009 - acc: 0.1643 - val_loss: 0.3513 - val_acc: 0.3720
Epoch 2/200
1/1 [==============================] - 0s 54ms/step - loss: 0.0946 - acc: 0.6714 - val_loss: 0.3403 - val_acc: 0.5160
Epoch 3/200
1/1 [==============================] - 0s 38ms/step - loss: 0.0884 - acc: 0.9071 - val_loss: 0.3295 - val_acc: 0.6020
Epoch 4/200
1/1 [==============================] - 0s 39ms/step - loss: 0.0824 - acc: 0.9571 - val_loss: 0.3192 - val_acc: 0.6420
Epoch 5/200
1/1 [==============================] - 0s 58ms/step - loss: 0.0768 - acc: 0.9643 - val_loss: 0.3091 - val_acc: 0.6640
Epoch 6/200
1/1 [==============================] - 0s 85ms/step - loss: 0.0714 - acc: 0.9643 - val_loss: 0.2994 - val_acc: 0.6700
Epoch 7/200
1/1 [==============================] - 0s 134ms/step - loss: 0.0663 - acc: 0.9714 - val_loss: 0.2900 - val_acc: 0.6840
Epoch 8/200
1/1 [==============================] - 0s 101ms/step - loss: 0.0615 - acc: 0

##### Evaluate

In [54]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=mask_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 64ms/step - loss: 0.2652 - acc: 0.7840
Done.
Test loss: 0.26516053080558777
Test accuracy: 0.7839999794960022


## Citeseer

In [55]:
learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "citeseer"
tf.random.set_seed(seed=seed)

#### GCN Model

In [56]:
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

Pre-processing node features


/usr/local/lib/python3.10/dist-packages/spektral/datasets/citation.py:194: RuntimeWarning: divide by zero encountered in power
  r_inv = np.power(rowsum, -1).flatten()
/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


##### Train Model

In [57]:
model, weights_tr, weights_va, weights_te = create_GCN_text_classification(dataset, learning_rate)
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 3s 3s/step - loss: 1.7995 - acc: 0.2167 - val_loss: 1.7955 - val_acc: 0.4800
Epoch 2/200
1/1 [==============================] - 1s 534ms/step - loss: 1.7932 - acc: 0.5250 - val_loss: 1.7919 - val_acc: 0.5300
Epoch 3/200
1/1 [==============================] - 0s 361ms/step - loss: 1.7874 - acc: 0.6667 - val_loss: 1.7889 - val_acc: 0.5200
Epoch 4/200
1/1 [==============================] - 0s 334ms/step - loss: 1.7810 - acc: 0.7167 - val_loss: 1.7866 - val_acc: 0.5380
Epoch 5/200
1/1 [==============================] - 0s 398ms/step - loss: 1.7736 - acc: 0.7333 - val_loss: 1.7848 - val_acc: 0.5500
Epoch 6/200
1/1 [==============================] - 0s 487ms/step - loss: 1.7677 - acc: 0.7667 - val_loss: 1.7831 - val_acc: 0.5660
Epoch 7/200
1/1 [==============================] - 0s 442ms/step - loss: 1.7601 - acc: 0.8000 - val_loss: 1.7815 - val_acc: 0.5900
Epoch 8/200
1/1 [==============================] - 1s 501ms/step - loss: 1.7574 - acc:

##### Evaluate

In [58]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 76ms/step - loss: 1.2878 - acc: 0.7070
Done.
Test loss: 1.287792682647705
Test accuracy: 0.7070000171661377


#### Simplify GCN Model

In [59]:
K = 2  # Propagation steps for SGCN
dataset = Citation("citeseer", transforms=[LayerPreprocess(GCNConv), SGCN(K)])
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


##### Train model

In [60]:
model = create_SGCN_text_classification(dataset, learning_rate)

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 3703)]               0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 3327)]               0         []                            
                                                                                                  
 gcn_conv_17 (GCNConv)       (None, 6)                    22218     ['input_7[0][0]',             
                                                                     'input_8[0][0]']             
                                                                                                  
Total params: 22218 (86.79 KB)
Trainable params: 22218 (86.79 KB)
Non-trainable params: 0 (0

In [61]:
loader_tr = SingleLoader(dataset, sample_weights=mask_tr)
loader_va = SingleLoader(dataset, sample_weights=mask_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 1s 689ms/step - loss: 0.0653 - acc: 0.1167 - val_loss: 0.2595 - val_acc: 0.4400
Epoch 2/200
1/1 [==============================] - 0s 105ms/step - loss: 0.0573 - acc: 0.8583 - val_loss: 0.2484 - val_acc: 0.5900
Epoch 3/200
1/1 [==============================] - 0s 90ms/step - loss: 0.0498 - acc: 0.9500 - val_loss: 0.2378 - val_acc: 0.6360
Epoch 4/200
1/1 [==============================] - 0s 114ms/step - loss: 0.0430 - acc: 0.9500 - val_loss: 0.2279 - val_acc: 0.6380
Epoch 5/200
1/1 [==============================] - 0s 95ms/step - loss: 0.0370 - acc: 0.9500 - val_loss: 0.2187 - val_acc: 0.6500
Epoch 6/200
1/1 [==============================] - 0s 91ms/step - loss: 0.0318 - acc: 0.9500 - val_loss: 0.2103 - val_acc: 0.6480
Epoch 7/200
1/1 [==============================] - 0s 96ms/step - loss: 0.0273 - acc: 0.9500 - val_loss: 0.2027 - val_acc: 0.6560
Epoch 8/200
1/1 [==============================] - 0s 87ms/step - loss: 0.0236 - acc: 0

##### Evaluate

In [62]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=mask_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 38ms/step - loss: 0.3097 - acc: 0.6630
Done.
Test loss: 0.3096916973590851
Test accuracy: 0.6629999876022339


## Pubmed

In [63]:
learning_rate = 1e-2
seed = 0
epochs = 200
patience = 10
data = "pubmed"
tf.random.set_seed(seed=seed)

#### GCN Model

In [64]:
dataset = Citation(data, normalize_x=True, transforms=[LayerPreprocess(GCNConv)])

Pre-processing node features


/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


##### Train Model

In [65]:
model, weights_tr, weights_va, weights_te = create_GCN_text_classification(dataset, learning_rate)
loader_tr = SingleLoader(dataset, sample_weights=weights_tr)
loader_va = SingleLoader(dataset, sample_weights=weights_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1.1053 - acc: 0.4167 - val_loss: 1.1031 - val_acc: 0.6200
Epoch 2/200
1/1 [==============================] - 0s 179ms/step - loss: 1.1022 - acc: 0.6167 - val_loss: 1.1001 - val_acc: 0.6900
Epoch 3/200
1/1 [==============================] - 0s 176ms/step - loss: 1.0964 - acc: 0.8000 - val_loss: 1.0967 - val_acc: 0.7220
Epoch 4/200
1/1 [==============================] - 0s 157ms/step - loss: 1.0910 - acc: 0.8667 - val_loss: 1.0930 - val_acc: 0.7300
Epoch 5/200
1/1 [==============================] - 0s 153ms/step - loss: 1.0829 - acc: 0.8667 - val_loss: 1.0891 - val_acc: 0.7440
Epoch 6/200
1/1 [==============================] - 0s 153ms/step - loss: 1.0783 - acc: 0.8167 - val_loss: 1.0850 - val_acc: 0.7440
Epoch 7/200
1/1 [==============================] - 0s 156ms/step - loss: 1.0681 - acc: 0.8500 - val_loss: 1.0807 - val_acc: 0.7340
Epoch 8/200
1/1 [==============================] - 0s 152ms/step - loss: 1.0620 - acc:

##### Evaluate

In [66]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=weights_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 38ms/step - loss: 0.7147 - acc: 0.7910
Done.
Test loss: 0.7146878838539124
Test accuracy: 0.7909999489784241


#### SimpleGCN Model

In [67]:
K = 2  # Propagation steps for SGCN
dataset = Citation("pubmed", transforms=[LayerPreprocess(GCNConv), SGCN(K)])
mask_tr, mask_va, mask_te = dataset.mask_tr, dataset.mask_va, dataset.mask_te

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


##### Train model

In [68]:
model = create_SGCN_text_classification(dataset, learning_rate)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 500)]                0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, 19717)]              0         []                            
                                                                                                  
 gcn_conv_20 (GCNConv)       (None, 3)                    1500      ['input_9[0][0]',             
                                                                     'input_10[0][0]']            
                                                                                                  
Total params: 1500 (5.86 KB)
Trainable params: 1500 (5.86 KB)
Non-trainable params: 0 (0.00 

In [69]:
loader_tr = SingleLoader(dataset, sample_weights=mask_tr)
loader_va = SingleLoader(dataset, sample_weights=mask_va)
model.fit(
    loader_tr.load(),
    steps_per_epoch=loader_tr.steps_per_epoch,
    validation_data=loader_va.load(),
    validation_steps=loader_va.steps_per_epoch,
    epochs=epochs,
    callbacks=[EarlyStopping(patience=patience, restore_best_weights=True)],
)

Epoch 1/200
1/1 [==============================] - 1s 628ms/step - loss: 0.0034 - acc: 0.3167 - val_loss: 0.0279 - val_acc: 0.3520
Epoch 2/200
1/1 [==============================] - 0s 85ms/step - loss: 0.0034 - acc: 0.4500 - val_loss: 0.0278 - val_acc: 0.4520
Epoch 3/200
1/1 [==============================] - 0s 86ms/step - loss: 0.0033 - acc: 0.5667 - val_loss: 0.0277 - val_acc: 0.5160
Epoch 4/200
1/1 [==============================] - 0s 85ms/step - loss: 0.0033 - acc: 0.6667 - val_loss: 0.0277 - val_acc: 0.6040
Epoch 5/200
1/1 [==============================] - 0s 127ms/step - loss: 0.0033 - acc: 0.7500 - val_loss: 0.0276 - val_acc: 0.6300
Epoch 6/200
1/1 [==============================] - 0s 97ms/step - loss: 0.0033 - acc: 0.7500 - val_loss: 0.0275 - val_acc: 0.6560
Epoch 7/200
1/1 [==============================] - 0s 84ms/step - loss: 0.0033 - acc: 0.8167 - val_loss: 0.0275 - val_acc: 0.6760
Epoch 8/200
1/1 [==============================] - 0s 89ms/step - loss: 0.0033 - acc: 0.

##### Evaluate

In [70]:
print("Evaluating model.")
loader_te = SingleLoader(dataset, sample_weights=mask_te)
eval_results = model.evaluate(loader_te.load(), steps=loader_te.steps_per_epoch)
print("Done.\n" "Test loss: {}\n" "Test accuracy: {}".format(*eval_results))

Evaluating model.
1/1 [==============================] - 0s 43ms/step - loss: 0.0499 - acc: 0.7440
Done.
Test loss: 0.04988781362771988
Test accuracy: 0.7440000176429749
